In [4]:
import pandas as pd
import psycopg2
def fetch_futures(cursor, symbol, x=12):
    query = f'''
        SELECT *
        FROM ohlcv_future_per_minute ofpm 
        WHERE ofpm.symbol = '{symbol}'
        AND ofpm.expiry_type = 'I'
        AND ofpm.expiry = (
            SELECT ofpem.expiry 
            FROM ohlcv_future_per_minute ofpem 
            WHERE ofpem.symbol = '{symbol}'
            AND ofpem.expiry_type = 'I'
            GROUP BY ofpem.expiry 
            OFFSET {x} 
            LIMIT 1
        )
        ORDER BY date_timestamp ASC
    '''
    cursor.execute(query, (symbol, symbol, x))
    rows = cursor.fetchall()
    df = pd.DataFrame(rows, columns=[desc[0] for desc in cursor.description])
    return df

conn = psycopg2.connect(
           dbname="qdap_test",
           user="amt",
           password="your_password",
           host="192.168.2.23",
           port="5432"
       )

# Create a cursor object using the cursor() method
cursor = conn.cursor()
symbol = 'BANKNIFTY'
df = fetch_futures(cursor, symbol)
df['date_timestamp'] = pd.to_datetime(df['date_timestamp'])
df.set_index('id', inplace=True)

In [6]:
df.reset_index(drop=True , inplace=True)
df.head()



,symbol,date_timestamp,open,low,high,close,volume,expiry,expiry_type
0,BANKNIFTY,2023-04-28 09:15:00,4305995,4304000,4313995,4312890,62525,2023-05-25 14:30:00,I
1,BANKNIFTY,2023-04-28 09:16:00,4313000,4310355,4313270,4310500,23100,2023-05-25 14:30:00,I
2,BANKNIFTY,2023-04-28 09:17:00,4310855,4310360,4312300,4310650,16600,2023-05-25 14:30:00,I
3,BANKNIFTY,2023-04-28 09:18:00,4310580,4309845,4311095,4309865,18875,2023-05-25 14:30:00,I
4,BANKNIFTY,2023-04-28 09:19:00,4310000,4304300,4310200,4304970,26400,2023-05-25 14:30:00,I
